In [1]:
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians


%pylab inline
%config InlineBackend.figure_formats = ['retina']

import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import math

sns.set()
pd.set_option('display.max_columns', None)

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/opt/anaconda3/envs/basics310/lib/python3.10/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['sin', 'sqrt', 'radians', 'cos']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [2]:
def distance(lat1, lon1, lat2, lon2):

    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    return distance

In [3]:

# Load your dataset into a pandas DataFrame
df = pd.read_csv('/Users/masayasugimotohomefolder/Desktop/MarchMadness/0000_completed_dataset_2023.csv')
df

game_id  team1_id  team2_id  season  team1_seed  team2_seed  \
0     2023-1345-1266      1345      1266    2023           1           2   
1     2023-1345-1243      1345      1243    2023           1           3   
2     2023-1345-1397      1345      1397    2023           1           4   
3     2023-1345-1181      1345      1181    2023           1           5   
4     2023-1345-1246      1345      1246    2023           1           6   
...              ...       ...       ...     ...         ...         ...   
2273  2023-1233-1421      1233      1421    2023          13          15   
2274  2023-1233-1224      1233      1224    2023          13          16   
2275  2023-1213-1421      1213      1421    2023          14          15   
2276  2023-1213-1224      1213      1224    2023          14          16   
2277  2023-1421-1224      1421      1224    2023          15          16   

     team1_position team2_position  strongseed  weakseed team1_region  \
0               W01            W02           1         2            W   
1               W01            W03           1         3            W   
2               W01            W04           1         4            W   
3               W01            W05           1         5            W   
4               W01            W06           1         6            W   
...             ...            ...         ...       ...          ...   
2273            Z13            Z15          13        15            Z   
2274            Z13            Z16          13        16            Z   
2275            Z14            Z15          14        15            Z   
2276            Z14            Z16          14        16            Z   
2277            Z15            Z16          15        16            Z   

     team2_region  slot       host  host_lat  host_long team1_teamname  \
0               W  R4W1   new_york   40.7127   -74.0059         Purdue   
1               W  R4W1   new_york   40.7127   -74.0059         Purdue   
2               W  R3W1   new_york   40.7127   -74.0059         Purdue   
3               W  R3W1   new_york   40.7127   -74.0059         Purdue   
4               W  R4W1   new_york   40.7127   -74.0059         Purdue   
...           ...   ...        ...       ...        ...            ...   
2273            Z  R4Z1  las_vegas   36.1700   115.1400           Iona   
2274            Z  R3Z1  las_vegas   36.1700   115.1400           Iona   
2275            Z  R3Z2  las_vegas   36.1700   115.1400   Grand Canyon   
2276            Z  R4Z1  las_vegas   36.1700   115.1400   Grand Canyon   
2277            Z  R4Z1  las_vegas   36.1700   115.1400  UNC Asheville   

     team2_teamname  team1_coach_id team1_coach_name  team1_pt_school_ncaa  \
0         Marquette  matt-painter-1     Matt Painter                    14   
1         Kansas St  matt-painter-1     Matt Painter                    14   
2         Tennessee  matt-painter-1     Matt Painter                    14   
3              Duke  matt-painter-1     Matt Painter                    14   
4          Kentucky  matt-painter-1     Matt Painter                    14   
...             ...             ...              ...                   ...   
2273  UNC Asheville   rick-pitino-1      Rick Pitino                     2   
2274         Howard   rick-pitino-1      Rick Pitino                     2   
2275  UNC Asheville    bryce-drew-1       Bryce Drew                     2   
2276         Howard    bryce-drew-1       Bryce Drew                     2   
2277         Howard  mike-morrell-1     Mike Morrell                     1   

      team1_pt_overall_ncaa  team1_pt_school_s16  team1_pt_overall_s16  \
0                        15                    6                     6   
1                        15                    6                     6   
2                        15                    6                     6   
3                        15                    6                     6   
4                        15

In [4]:
df['team1_exp'] = (df['team1_adjoe']**11.5)/ ((df['team1_adjde']**11.5)+(df['team1_adjoe']**11.5))
df['team2_exp'] = (df['team2_adjoe']**11.5)/ ((df['team2_adjde']**11.5)+(df['team2_adjoe']**11.5))


In [5]:
df

game_id  team1_id  team2_id  season  team1_seed  team2_seed  \
0     2023-1345-1266      1345      1266    2023           1           2   
1     2023-1345-1243      1345      1243    2023           1           3   
2     2023-1345-1397      1345      1397    2023           1           4   
3     2023-1345-1181      1345      1181    2023           1           5   
4     2023-1345-1246      1345      1246    2023           1           6   
...              ...       ...       ...     ...         ...         ...   
2273  2023-1233-1421      1233      1421    2023          13          15   
2274  2023-1233-1224      1233      1224    2023          13          16   
2275  2023-1213-1421      1213      1421    2023          14          15   
2276  2023-1213-1224      1213      1224    2023          14          16   
2277  2023-1421-1224      1421      1224    2023          15          16   

     team1_position team2_position  strongseed  weakseed team1_region  \
0               W01            W02           1         2            W   
1               W01            W03           1         3            W   
2               W01            W04           1         4            W   
3               W01            W05           1         5            W   
4               W01            W06           1         6            W   
...             ...            ...         ...       ...          ...   
2273            Z13            Z15          13        15            Z   
2274            Z13            Z16          13        16            Z   
2275            Z14            Z15          14        15            Z   
2276            Z14            Z16          14        16            Z   
2277            Z15            Z16          15        16            Z   

     team2_region  slot       host  host_lat  host_long team1_teamname  \
0               W  R4W1   new_york   40.7127   -74.0059         Purdue   
1               W  R4W1   new_york   40.7127   -74.0059         Purdue   
2               W  R3W1   new_york   40.7127   -74.0059         Purdue   
3               W  R3W1   new_york   40.7127   -74.0059         Purdue   
4               W  R4W1   new_york   40.7127   -74.0059         Purdue   
...           ...   ...        ...       ...        ...            ...   
2273            Z  R4Z1  las_vegas   36.1700   115.1400           Iona   
2274            Z  R3Z1  las_vegas   36.1700   115.1400           Iona   
2275            Z  R3Z2  las_vegas   36.1700   115.1400   Grand Canyon   
2276            Z  R4Z1  las_vegas   36.1700   115.1400   Grand Canyon   
2277            Z  R4Z1  las_vegas   36.1700   115.1400  UNC Asheville   

     team2_teamname  team1_coach_id team1_coach_name  team1_pt_school_ncaa  \
0         Marquette  matt-painter-1     Matt Painter                    14   
1         Kansas St  matt-painter-1     Matt Painter                    14   
2         Tennessee  matt-painter-1     Matt Painter                    14   
3              Duke  matt-painter-1     Matt Painter                    14   
4          Kentucky  matt-painter-1     Matt Painter                    14   
...             ...             ...              ...                   ...   
2273  UNC Asheville   rick-pitino-1      Rick Pitino                     2   
2274         Howard   rick-pitino-1      Rick Pitino                     2   
2275  UNC Asheville    bryce-drew-1       Bryce Drew                     2   
2276         Howard    bryce-drew-1       Bryce Drew                     2   
2277         Howard  mike-morrell-1     Mike Morrell                     1   

      team1_pt_overall_ncaa  team1_pt_school_s16  team1_pt_overall_s16  \
0                        15                    6                     6   
1                        15                    6                     6   
2                        15                    6                     6   
3                        15                    6                     6   
4                        15

In [6]:
# List of team IDs to modify
team_id_list = [1417, 1274,1397]

# Loop through each team ID and modify the relevant columns
for team_id in team_id_list:
    # Boolean indexing to select rows with team1_id = team_id
    mask1 = df['team1_id'] == team_id

    # Subtract 10% from the values in team1_herop_percentot column for the selected rows
    df.loc[mask1, 'team1_herop_percentot'] *= 0.8
    df.loc[mask1, 'team1_exp'] *= 0.9

    # Boolean indexing to select rows with team2_id = team_id
    mask2 = df['team2_id'] == team_id

    # Subtract 10% from the values in team2_herop_percentot column for the selected rows
    df.loc[mask2, 'team2_herop_percentot'] *= 0.8
    df.loc[mask2, 'team2_exp'] *= 0.9

# Calculate the log5 for all teams
df['log5'] = (df['team1_exp'] - (df['team1_exp']*df['team2_exp']))/ (df['team1_exp']+df['team2_exp']-(2*df['team1_exp']*df['team2_exp']))


In [7]:

# Define the list of features to calculate the difference for
features = ["dist", "seed", "lat", "long", "pt_school_ncaa", "pt_overall_ncaa", "pt_school_s16", "pt_overall_s16", "pt_school_ff", "pt_overall_ff", "pt_career_school_wins", "pt_career_school_losses", "pt_career_overall_wins", "pt_career_overall_losses", "pt_team_season_wins", "pt_team_season_losses", "pt_coach_season_wins", "pt_coach_season_losses", "fg2pct", "fg3pct", "ftpct", "blockpct", "oppfg2pct", "oppfg3pct", "oppftpct", "oppblockpct", "f3grate", "oppf3grate", "arate", "opparate", "stlrate", "oppstlrate", "tempo", "adjtempo", "oe", "adjoe", "de", "adjde", 'opexpperpart_men','opexpperteam_men', 'rev_men','exp_men','herop_percentot', 'exp']
#"ap_final", "ap_preseason", "coaches_before_final", "coaches_preseason" has been removed


# calculate the distance between host and team1 for each row
df['team1_dist'] = df.apply(lambda row: distance(row['host_lat'], row['host_long'], row['team1_lat'], row['team1_long']), axis=1)

# calculate the distance between host and team2 for each row
df['team2_dist'] = df.apply(lambda row: distance(row['host_lat'], row['host_long'], row['team2_lat'], row['team2_long']), axis=1)

df = df.select_dtypes(include='number')
df = df.drop(columns=['team1_id', 'team2_id', 'strongseed', 'weakseed', 'season', 'host_lat', 'host_long'])

# Use list comprehension to create new columns for the difference between team1 and team2
df[[f"{feature}_diff" for feature in features]] = df[[f"team1_{feature}" for feature in features]].values - df[[f"team2_{feature}" for feature in features]].values

# Use list comprehension to drop the original team1 and team2 columns
df = df[[col for col in df.columns if not col.startswith("team1_") and not col.startswith("team2_")]]

# create a boolean mask of NaN values
mask = df.isnull()

# identify columns with at least one NaN value
cols_with_nan = mask.any()

# extract column names with NaN values
cols_with_nan_names = cols_with_nan[cols_with_nan == True].index.tolist()

# remove columns with NaN values
df = df.drop(columns=cols_with_nan_names)
df = df.drop(columns=['lat_diff', 'long_diff'])


# print updated dataframe
df

log5   dist_diff  seed_diff  pt_school_ncaa_diff  \
0     0.580230  -90.092181       -1.0                 12.0   
1     0.646711 -836.937663       -2.0                 13.0   
2     0.712626   76.340702       -3.0                  9.0   
3     0.645139  412.173087       -4.0                 13.0   
4     0.680726  139.493545       -5.0                  3.0   
...        ...         ...        ...                  ...   
2273  0.753330 -431.457697       -2.0                  1.0   
2274  0.836851 -178.664431       -3.0                  1.0   
2275  0.613694 -981.999904       -1.0                  1.0   
2276  0.727385 -729.206637       -2.0                  1.0   
2277  0.626804  252.793266       -1.0                  0.0   

      pt_overall_ncaa_diff  pt_school_s16_diff  pt_overall_s16_diff  \
0                      5.0                 6.0                  5.0   
1                     14.0                 6.0                  6.0   
2                    -12.0                 5.0                 -1.0   
3                     14.0                 6.0                  6.0   
4                     -7.0                -2.0                 -9.0   
...                    ...                 ...                  ...   
2273                  22.0                 0.0                 13.0   
2274                  22.0                 0.0                 13.0   
2275                   4.0                 0.0                  0.0   
2276                   4.0                 0.0                  0.0   
2277                   0.0                 0.0                  0.0   

      pt_school_ff_diff  pt_overall_ff_diff  pt_career_school_wins_diff  \
0                   0.0                -1.0                       366.0   
1                   0.0                 0.0                       390.0   
2                   0.0                -1.0                       240.0   
3                   0.0                 0.0                       387.0   
4                  -4.0                -6.0                        27.0   
...                 ...                 ...                         ...   
2273                0.0                 7.0                        -9.0   
2274                0.0                 7.0                        21.0   
2275                0.0                 0.0                        -9.0   
2276                0.0                 0.0                        21.0   
2277                0.0                 0.0                        30.0   

      pt_career_school_losses_diff  pt_career_overall_wins_diff  \
0                            178.0                        119.0   
1                            188.0                        415.0   
2                            106.0                       -339.0   
3                            189.0                        412.0   
4                             85.0                       -393.0   
...                            ...                          ...   
2273                         -54.0                        761.0   
2274                         -37.0                        791.0   
2275                         -49.0                        155.0   
2276                         -32.0                        185.0   
2277                          17.0                         30.0   

      pt_career_overall_losses_diff  pt_team_season_wins_diff  \
0                              41.0                       1.0   
1                             193.0                       6.0   
2                            -203.0                       6.0   
3                             194.0                       3.0   
4                             -50.0                       8.0   
...                             ...                       ...   
2273                          217.0                       0.0   
2274                          234.0                       5.0   
2275                           59.0                      -3.0   
2276                           76.0                     

In [8]:
df = df.rename(columns={'dist_diff': 'diff_dist'})

In [9]:
df.to_csv('0001_2023_dataset_difference_all_features_injured.csv', index=False)

In [10]:
columns_to_extract = [
 'diff_dist',
 'seed_diff',
#  'pt_career_school_wins_diff',
#  'pt_career_overall_wins_diff',
 'pt_team_season_wins_diff',
#  'adjoe_diff',
#  'adjde_diff',
 'pt_school_ff_diff',
 'pt_overall_ncaa_diff',
 'pt_school_ncaa_diff',
 'arate_diff',
 'fg3pct_diff',
#  'opexpperpart_men_diff',
#  'opexpperteam_men_diff',
 'rev_men_diff',
 'exp_men_diff',
 'herop_percentot_diff',
 'log5'
#  'team1_won'
]


df_extracted = df.loc[:, columns_to_extract]

In [11]:
df_extracted.to_csv('0001_2023_dataset_difference_injured.csv', index=False)

In [12]:
df_extracted

diff_dist  seed_diff  pt_team_season_wins_diff  pt_school_ff_diff  \
0     -90.092181       -1.0                       1.0                0.0   
1    -836.937663       -2.0                       6.0                0.0   
2      76.340702       -3.0                       6.0                0.0   
3     412.173087       -4.0                       3.0                0.0   
4     139.493545       -5.0                       8.0               -4.0   
...          ...        ...                       ...                ...   
2273 -431.457697       -2.0                       0.0                0.0   
2274 -178.664431       -3.0                       5.0                0.0   
2275 -981.999904       -1.0                      -3.0                0.0   
2276 -729.206637       -2.0                       2.0                0.0   
2277  252.793266       -1.0                       5.0                0.0   

      pt_overall_ncaa_diff  pt_school_ncaa_diff  arate_diff  fg3pct_diff  \
0                      5.0                 12.0    1.540546    -2.161705   
1                     14.0                 13.0   -3.036958    -1.393867   
2                    -12.0                  9.0   -5.267315    -0.312735   
3                     14.0                 13.0    4.250647    -0.937504   
4                     -7.0                  3.0    5.243037    -2.719303   
...                    ...                  ...         ...          ...   
2273                  22.0                  1.0    3.277714    -2.854469   
2274                  22.0                  1.0   -1.406933    -1.227293   
2275                   4.0                  1.0   -2.981360    -1.283735   
2276                   4.0                  1.0   -7.666007     0.343440   
2277                   0.0                  0.0   -4.684647     1.627176   

      rev_men_diff  exp_men_diff  herop_percentot_diff      log5  
0       -2928307.0    -4851526.0              0.110291  0.580230  
1        2505243.0       81648.0              0.064352  0.646711  
2       -6947613.0    -7759872.0              0.155648  0.712626  
3      -13391495.0    -7886602.0              0.090267  0.645139  
4       -8525088.0   -11041000.0              0.080805  0.680726  
...            ...           ...                   ...       ...  
2273     2701935.0     2701935.0             -0.064651  0.753330  
2274     1092889.0     2385794.0              0.046512  0.836851  
2275     4373920.0     4373920.0             -0.044380  0.613694  
2276     2764874.0     4057779.0              0.066783  0.727385  
2277    -1609046.0     -316141.0              0.111163  0.626804  

[2278 rows x 12 columns]